# Expected:
- Takes one Array of seven pictures
- calculates embedding of each picture via CNN
- returns embeddings either via array or concatenated

In [1]:
import os
import pandas as pd

from torchvision.io import read_image, ImageReadMode

In [2]:
path_data = "../data/"

metadata_df = pd.read_csv(os.path.join(path_data, "HandInfo.csv"))
metadata_df.head(20)

,id,age,gender,skinColor,accessories,nailPolish,aspectOfHand,imageName,irregularities
0,0,27,male,fair,0,0,dorsal right,Hand_0000002.jpg,0
1,0,27,male,fair,0,0,dorsal right,Hand_0000003.jpg,0
2,0,27,male,fair,0,0,dorsal right,Hand_0000004.jpg,0
3,0,27,male,fair,0,0,dorsal right,Hand_0000005.jpg,0
4,0,27,male,fair,0,0,dorsal right,Hand_0000006.jpg,0
5,0,27,male,fair,0,0,dorsal right,Hand_0000007.jpg,0
6,0,27,male,fair,0,0,dorsal right,Hand_0000008.jpg,0
7,0,27,male,fair,0,0,dorsal right,Hand_0000009.jpg,0
8,0,27,male,fair,0,0,dorsal right,Hand_0000010.jpg,0
9,0,27,male,fair,0,0,dorsal right,Hand_0000011.jpg,0


In [ ]:
path_to_images = "../data/Hands/Hands/"

# Pick 9 images
ids = [0, 20, 40, 60, 80, 100, 120, 140, 160]
random_images = metadata_df.imageName[ids]
print(random_images.size)

for image in random_images:
    # Construct the full path to the image file
    img_path = os.path.join(path_to_images, image)
    print(img_path)
    # Read the image file
    image = read_image(img_path, ImageReadMode.RGB)
    # Der Output ist ein Tensor mit den RGB-Werten
    # the output is a tensor with rgb-values
    # this tensor is being transformed before it gets embedded by the CNN
    #print(image)

9
../data/Hands/Hands/Hand_0000002.jpg
tensor([[[244, 243, 243,  ..., 255, 255, 255],
         [243, 243, 242,  ..., 255, 255, 255],
         [244, 244, 243,  ..., 255, 255, 255],
         ...,
         [255, 255, 255,  ..., 255, 255, 255],
         [255, 255, 255,  ..., 255, 255, 255],
         [255, 255, 255,  ..., 255, 255, 255]],

        [[255, 255, 255,  ..., 255, 255, 255],
         [255, 255, 254,  ..., 255, 255, 255],
         [255, 255, 255,  ..., 255, 255, 255],
         ...,
         [255, 255, 255,  ..., 255, 255, 255],
         [255, 255, 255,  ..., 255, 255, 255],
         [255, 255, 255,  ..., 255, 255, 255]],

        [[254, 253, 253,  ..., 255, 255, 255],
         [253, 253, 252,  ..., 255, 255, 255],
         [254, 254, 253,  ..., 255, 255, 255],
         ...,
         [255, 255, 255,  ..., 255, 255, 255],
         [255, 255, 255,  ..., 255, 255, 255],
         [255, 255, 255,  ..., 255, 255, 255]]], dtype=torch.uint8)
../data/Hands/Hands/Hand_0000025.jpg
tensor([[[2

In [4]:
# workaround to import python modules into the notebook from another directory
import sys
sys.path.insert(1, '../')

import embeddings_utils
from embeddings_utils import choose_device, build_densenet_model, load_image, calculate_embedding, calculate_embeddings, calculate_concatenated_embedding

In [5]:
# test if module and functions are being recognized
choose_device()

device(type='cpu')

In [6]:
# test if densenet model is buing built and classifier is being replaced with an Identity Layer
build_densenet_model()

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [7]:
# test if one image results in an embedding of 1024 dimensions
test_image = random_images[0]
embeddings_utils.preprocess_image(load_image(test_image, path_to_images))


Image([[[0.9569, 0.9569, 0.9647,  ..., 1.0000, 1.0000, 1.0000],
        [0.9647, 0.9686, 0.9725,  ..., 1.0000, 1.0000, 1.0000],
        [0.9765, 0.9765, 0.9843,  ..., 1.0000, 1.0000, 1.0000],
        ...,
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000]],

       [[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        [0.9961, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        ...,
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000]],

       [[0.9922, 0.9961, 0.9961,  ..., 1.0000, 1.0000, 1.0000],
        [0.9961, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        [0.9961, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        ..

tensor([[[[0.9569, 0.9569, 0.9647,  ..., 1.0000, 1.0000, 1.0000],
          [0.9647, 0.9686, 0.9725,  ..., 1.0000, 1.0000, 1.0000],
          [0.9765, 0.9765, 0.9843,  ..., 1.0000, 1.0000, 1.0000],
          ...,
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000]],

         [[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [0.9961, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          ...,
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000]],

         [[0.9922, 0.9961, 0.9961,  ..., 1.0000, 1.0000, 1.0000],
          [0.9961, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [0.9961, 1.0000, 1.0000,  ..., 1

In [8]:
tensor = calculate_embedding(test_image, build_densenet_model(), path_to_images)

tensor.shape

Image([[[0.9569, 0.9569, 0.9647,  ..., 1.0000, 1.0000, 1.0000],
        [0.9647, 0.9686, 0.9725,  ..., 1.0000, 1.0000, 1.0000],
        [0.9765, 0.9765, 0.9843,  ..., 1.0000, 1.0000, 1.0000],
        ...,
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000]],

       [[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        [0.9961, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        ...,
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000]],

       [[0.9922, 0.9961, 0.9961,  ..., 1.0000, 1.0000, 1.0000],
        [0.9961, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        [0.9961, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        ..

torch.Size([1, 1024])

In [9]:
tensor[0]

tensor([2.2893e-04, 2.9098e-03, 5.7072e-03,  ..., 3.5405e+00, 1.1242e-02,
        9.9475e-02])

In [10]:
random_images.size

9

In [11]:
# test if nine images result in nine embedding vectors
calculate_embeddings(random_images, path_to_images)

Image([[[0.9569, 0.9569, 0.9647,  ..., 1.0000, 1.0000, 1.0000],
        [0.9647, 0.9686, 0.9725,  ..., 1.0000, 1.0000, 1.0000],
        [0.9765, 0.9765, 0.9843,  ..., 1.0000, 1.0000, 1.0000],
        ...,
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000]],

       [[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        [0.9961, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        ...,
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000]],

       [[0.9922, 0.9961, 0.9961,  ..., 1.0000, 1.0000, 1.0000],
        [0.9961, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        [0.9961, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        ..

[tensor([[2.2893e-04, 2.9098e-03, 5.7072e-03,  ..., 3.5405e+00, 1.1242e-02,
          9.9475e-02]]),
 tensor([[2.4183e-04, 2.5153e-03, 5.5145e-03,  ..., 3.1323e+00, 8.3295e-02,
          1.7921e-01]]),
 tensor([[2.3955e-04, 2.1291e-03, 5.8599e-03,  ..., 2.5442e+00, 5.7988e-03,
          2.7524e-01]]),
 tensor([[2.2107e-04, 2.4643e-03, 6.3350e-03,  ..., 3.4737e+00, 2.5360e-03,
          1.1454e-01]]),
 tensor([[1.8347e-04, 1.5863e-03, 5.7568e-03,  ..., 5.9341e+00, 4.7884e-02,
          2.5560e-01]]),
 tensor([[2.3652e-04, 8.5687e-04, 5.5613e-03,  ..., 2.6770e+00, 9.6306e-02,
          1.6008e-01]]),
 tensor([[2.6193e-04, 1.2095e-03, 6.0958e-03,  ..., 2.0814e+00, 5.4815e-03,
          2.5568e-01]]),
 tensor([[2.0418e-04, 2.2577e-03, 6.6160e-03,  ..., 2.6105e+00, 7.5762e-03,
          4.1048e-01]]),
 tensor([[2.5064e-04, 1.5085e-03, 7.1963e-03,  ..., 3.6506e+00, 1.7747e-01,
          1.5082e-01]])]

In [12]:
# test if nine images result in one super embedding of the size of 9216
super_embedding_test = calculate_concatenated_embedding(random_images, path_to_images)

Image([[[0.9569, 0.9569, 0.9647,  ..., 1.0000, 1.0000, 1.0000],
        [0.9647, 0.9686, 0.9725,  ..., 1.0000, 1.0000, 1.0000],
        [0.9765, 0.9765, 0.9843,  ..., 1.0000, 1.0000, 1.0000],
        ...,
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000]],

       [[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        [0.9961, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        ...,
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000]],

       [[0.9922, 0.9961, 0.9961,  ..., 1.0000, 1.0000, 1.0000],
        [0.9961, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        [0.9961, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        ..

In [13]:
super_embedding_test

[0.00022893161803949624,
 0.0029098154045641422,
 0.005707153119146824,
 0.0026925646234303713,
 0.05869431421160698,
 0.819465696811676,
 0.00027159007731825113,
 0.004875057842582464,
 0.2569432556629181,
 0.00045091830543242395,
 0.0013945663813501596,
 0.019074302166700363,
 0.0007478640181943774,
 0.028698695823550224,
 0.02435864694416523,
 5.419785520643927e-05,
 0.0015508536016568542,
 0.7940622568130493,
 0.0040832385420799255,
 0.00025893974816426635,
 0.0032196182291954756,
 0.008546072989702225,
 0.00014228168583940715,
 0.000939486431889236,
 0.0018359085079282522,
 0.000759793387260288,
 0.0016405979404225945,
 0.012946780771017075,
 0.008070086129009724,
 0.0026198674459010363,
 0.006772476714104414,
 0.0014896831708028913,
 0.026246167719364166,
 0.00012889453500974923,
 0.256203293800354,
 0.0031423845794051886,
 0.01699080504477024,
 0.07733014225959778,
 0.0009471962694078684,
 0.08271808922290802,
 0.052490074187517166,
 0.0005078413523733616,
 0.099698506295681,
 0

In [14]:
len(super_embedding_test)
# erwartet: 9216

9216